In [6]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
from urllib.parse import urljoin
import pandas as pd
import re



In [ ]:
# Index range for all 34 matchdays each season
matchday_ind_ls = range(1, 35)

# Index range for all season years from 1994 (94/95) to 2022 (22/23)
season_ind_ls = range(1994, 2023)

# Index range for all 18 clubs
clubs_ind_ls = range(1,19)


# Base URL of Transfermarkt Webpage
BASE_URL = 'https://www.transfermarkt.de/'

# header config for Browser setup
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}


def CLUBS_OVERVIEW_URL(season):
    """
    Reads the url for overview of all clubs for specific season

    Args:
        season (int): Accept years in form of yyyy

    Returns:
        response object: Provides methods and attributes to access the data returned by the HTTP request
    """
    url = urljoin(BASE_URL, f'bundesliga/startseite/wettbewerb/L1/plus/?saison_id={season}')
    response = requests.get(url, headers=headers)
    print(url)
    return response

# Overview over all match results of each season and matchdays
def MATCH_RESULTS_URL(season, matchday):
    """
    Reads the url for all match results of a specific season and a specific matchday
    
    Args:
        season (int): Accept years in form of yyyy
        matchday (int): Accepts an index from 1 to 18 for the number of matchdays

    Returns:
        response object: Provides methods and attributes to access the data returned by the HTTP request
    """
    url = urljoin(BASE_URL, f'bundesliga/spieltag/wettbewerb/L1/plus/?saison_id={season}&spieltag={matchday}')
    response = requests.get(url, headers=headers)
    print(url)
    return response



def dict_to_df(col_name_ls, value_ls):
    """
    Reads the column names and their values in form of lists. Store it into one dictionary to convert it to a Dataframe format 
    
    Args:
        col_name_ls (list['str']): list of column names
        value_ls (list('list')): list of lists with the values for each column

    Returns:
        df (DataFrame):  
    """
    dict = {}
    for enum in range(len(col_name_ls)):
        dict[col_name_ls[enum]] = value_ls[enum]
    df = pd.DataFrame(dict)
    return df

# Clubs Overview over all seasons

In [2]:
# Loop through all season for clubs overview
df_clubs_overview = pd.DataFrame()
for season in season_ind_ls:
    response = CLUBS_OVERVIEW_URL(season)
    soup = BeautifulSoup(response.content, 'lxml')

    # Clubnames 
    clubs_ls = []
    clubs = soup.find('div', id='yw1').find_all('td', class_='hauptlink no-border-links')
    for club in clubs:
        clubs_ls.append(club.text.strip())

    # The number of players in the pool
    players_count_ls = []
    players_count = soup.find('div', id='yw1').find_all('td', class_ = 'zentriert')
    for player_count in players_count:
        for item in player_count.find_all('a'):
            if item.text != '':
                players_count_ls.append(item.text.strip())

    # Average age of all players within a club
    players_avg_age_ls = []
    for row in clubs_ind_ls:
        dom = etree.HTML(str(soup))
        value = dom.xpath(f"//*[@id='yw1']/table/tbody/tr[{row}]/td[4]")[0].text
        players_avg_age_ls.append(value.strip())
    players_avg_age_ls

    # Number of legionaries within a club
    legionaries_ls = []
    for row in clubs_ind_ls:
        dom = etree.HTML(str(soup))
        value = dom.xpath(f"//*[@id='yw1']/table/tbody/tr[{row}]/td[5]")[0].text
        legionaries_ls.append(value.strip())
    legionaries_ls

    # Average market value of each club
    avg_market_value_ls = []
    for row in clubs_ind_ls:
        dom = etree.HTML(str(soup))
        value = dom.xpath(f"//*[@id='yw1']/table/tbody/tr[{row}]/td[6]")[0].text
        avg_market_value_ls.append(value.strip())
    avg_market_value_ls

    # Total market value of each club
    total_market_values_ls =[]
    total_market_values = soup.find('table', class_='items').find_all('td', class_="rechts")
    for item1 in total_market_values:
        for item2 in item1.find_all('a'):
            total_market_values_ls.append(item2.text.strip())
    total_market_values_ls


    # Write all Information to a Dataframe
    col_name_ls = ['CLUB_NAME', 'PLAYERS_COUNT', 'PLAYERS_AVG_AGE', 'LEGIONARIES_COUNT', 'AVG_MARKET_VALUE', 'TOTAL_MARKET_VALUE']
    value_ls = [clubs_ls, players_count_ls, players_avg_age_ls, legionaries_ls, avg_market_value_ls, total_market_values_ls]
    df = dict_to_df(col_name_ls=col_name_ls, value_ls=value_ls) 

    # add season information as column to dataframe
    df['season'] = f"{season}/{season+1}"

    # Concat all dataframes of each season into one dataframe 
    df_clubs_overview = pd.concat([df_clubs_overview, df])

https://www.transfermarkt.de/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=1994
https://www.transfermarkt.de/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=1995
https://www.transfermarkt.de/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=1996
https://www.transfermarkt.de/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=1997
https://www.transfermarkt.de/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=1998
https://www.transfermarkt.de/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=1999
https://www.transfermarkt.de/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2000
https://www.transfermarkt.de/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2001
https://www.transfermarkt.de/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2002
https://www.transfermarkt.de/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2003
https://www.transfermarkt.de/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2004
https://www.transfermarkt.de/bundesliga/startseite/wet

In [3]:
# save dataframe to pickle
#df_clubs_overview.to_pickle("./df_clubs_overview.pkl")

In [4]:
# read data from pickle
df_clubs_overview = pd.read_pickle("./df_clubs_overview.pkl")

In [5]:
df_clubs_overview

,CLUB_NAME,PLAYERS_COUNT,PLAYERS_AVG_AGE,LEGIONARIES_COUNT,AVG_MARKET_VALUE,TOTAL_MARKET_VALUE,season
0,FC Schalke 04,28,"25,4",7,-,-,1994/1995
1,SV Werder Bremen,26,"28,6",5,-,-,1994/1995
2,1.FC Kaiserslautern,26,"27,3",5,-,-,1994/1995
3,Hamburger SV,29,"26,4",9,-,-,1994/1995
4,Bayer 05 Uerdingen,24,"25,1",6,-,-,1994/1995
...,...,...,...,...,...,...,...
13,1.FC Köln,39,"24,9",15,"2,97 Mio. €","115,65 Mio. €",2022/2023
14,Hertha BSC,41,"25,4",21,"2,29 Mio. €","94,00 Mio. €",2022/2023
15,SV Werder Bremen,33,"24,7",10,"2,71 Mio. €","89,50 Mio. €",2022/2023
16,FC Schalke 04,43,"26,4",22,"2,04 Mio. €","87,78 Mio. €",2022/2023


# Retrieve matchday data

In [13]:
def extract_betting_percentages(word: str):
    """Extracts the betting percentages for win home team, remis and win away team"""
    # Define the regular expression pattern
    pattern = r'([\d,]+) %'

    # Find all matches using the pattern
    matches = re.findall(pattern, word)

    # Convert the matched values to floats
    return [float(match.replace(',', '.')) for match in matches] 

In [14]:
def extract_club_standings(word: str, pattern: str, home: bool):
    """Extracts club and standing from given word"""
    # Find the match using the pattern
    match = re.match(pattern, word)

    if match:
        if home:
            number = match.group(1)
            club = match.group(2)
        else:
            club = match.group(1)
            number = match.group(2)
        
        return club, number
    else:
        print("No match found.")
        return None, None

In [124]:
def get_matchday_data(season: int, matchday: int):
    """Retrieve matchday data for given season and matchday"""
    
    response = MATCH_RESULTS_URL(season, matchday)
    soup = BeautifulSoup(response.content, 'lxml')
    
    # retrieve tipps for matchdays
    match_tipps = soup.find_all('tr', class_='no-border tm-user-tendenz')
    tipp_list = [tipp.text.strip() for tipp in match_tipps]
    tipps = [tuple(extract_betting_percentages(tipp)) for tipp in tipp_list]

    # retrieve all home teams for given matchday
    home_teams = soup.find_all('td', class_='rechts hauptlink no-border-rechts hide-for-small spieltagsansicht-vereinsname')
    home_list = [team.text.strip() for team in home_teams]
    pattern = r'^\((\d+)\.\)\s+(.+)$'
    home = [(extract_club_standings(team, pattern, home=True)) for team in home_list]

    # retrieve all guest teams for given matchday
    teams_away = soup.find_all('td', class_='hauptlink no-border-links no-border-rechts hide-for-small spieltagsansicht-vereinsname')
    away_list = [team.text.strip() for team in teams_away]
  
    pattern = r'^(.+)\s+\((\d+)\.\)$'
    away = []
    for team in away_list:
        club, number = extract_club_standings(team, pattern, home=False)
        away.append((club.replace('\t', ''), number))

    # retrieve match results
    match_results = soup.find_all('td', class_='zentriert hauptlink no-border-rechts no-border-links spieltagsansicht-ergebnis')
    results_list = [results.text.strip().split(":") for results in match_results]
    results = [tuple(results) for results in results_list]
 
    # retrieve match days
    pattern = re.compile(r'^/aktuell/waspassiertheute/aktuell/new/datum/.*')  #find all href that starts with '/aktuell/waspassiertheute/aktuell/new/datum/'
    match_days = soup.find_all('a', href=pattern)
    days_list = [day.text.strip() for day in match_days]
    days = [(day,) for day in days_list]

    # create dataframe with match data
    columns = ["HOME_TEAM", "PLACE_HOME_TEAM", "AWAY_TEAM", "PLACE_AWAY_TEAM","WIN_PERC_HOME", 
               "REMIS_PERC", "WIN_PERC_AWAY", "MATCH_RESULTS_HOME", "MATCH_RESULTS_AWAY", "MATCH_DAY"]
    matches = [t1 + t2 + t3 + t4 + t5 for t1, t2, t3, t4, t5 in zip(home, away, tipps, results, days)]
    df = pd.DataFrame(matches, columns=columns)
    df['season'] = season
    df['matchday'] = matchday
    return df

In [125]:
# Index range for all 34 matchdays each season
matchday_ind_ls = range(1, 35)

# Index range for all season years from 2010 (2010/11) to 2022 (22/23)
season_ind_ls = range(2010, 2023)

df_matchday_data = pd.DataFrame()

for season in season_ind_ls:
    for matchday in matchday_ind_ls:
        df_matchday_data = pd.concat([df_matchday_data, get_matchday_data(season, matchday)])

https://www.transfermarkt.de/bundesliga/spieltag/wettbewerb/L1/plus/?saison_id=2010&spieltag=1
https://www.transfermarkt.de/bundesliga/spieltag/wettbewerb/L1/plus/?saison_id=2010&spieltag=2
https://www.transfermarkt.de/bundesliga/spieltag/wettbewerb/L1/plus/?saison_id=2010&spieltag=3
https://www.transfermarkt.de/bundesliga/spieltag/wettbewerb/L1/plus/?saison_id=2010&spieltag=4
https://www.transfermarkt.de/bundesliga/spieltag/wettbewerb/L1/plus/?saison_id=2010&spieltag=5
https://www.transfermarkt.de/bundesliga/spieltag/wettbewerb/L1/plus/?saison_id=2010&spieltag=6
https://www.transfermarkt.de/bundesliga/spieltag/wettbewerb/L1/plus/?saison_id=2010&spieltag=7
https://www.transfermarkt.de/bundesliga/spieltag/wettbewerb/L1/plus/?saison_id=2010&spieltag=8
https://www.transfermarkt.de/bundesliga/spieltag/wettbewerb/L1/plus/?saison_id=2010&spieltag=9
https://www.transfermarkt.de/bundesliga/spieltag/wettbewerb/L1/plus/?saison_id=2010&spieltag=10
https://www.transfermarkt.de/bundesliga/spieltag/

In [126]:
# save dataframe to pickle
df_matchday_data.to_pickle("./df_matchday_data.pkl")

In [127]:
# read data from pickle
df_matchday_data = pd.read_pickle("./df_matchday_data.pkl")

In [129]:
df_matchday_data

,HOME_TEAM,PLACE_HOME_TEAM,AWAY_TEAM,PLACE_AWAY_TEAM,WIN_PERC_HOME,REMIS_PERC,WIN_PERC_AWAY,MATCH_RESULTS_HOME,MATCH_RESULTS_AWAY,MATCH_DAY,season,matchday
0,Bayern München,3,VfL Wolfsburg,15,89.0,9.5,1.6,2,1,20.08.2010,2010,1
1,TSG Hoffenheim,11,Werder Bremen,13,11.9,24.8,63.3,4,1,21.08.2010,2010,1
2,Bor. M'gladbach,16,1.FC Nürnberg,6,79.5,15.4,5.1,1,1,21.08.2010,2010,1
3,1.FC Köln,10,1.FC K'lautern,7,44.7,34.3,21.0,1,3,21.08.2010,2010,1
4,SC Freiburg,9,FC St. Pauli,18,39.8,37.1,23.1,1,3,21.08.2010,2010,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4,Bor. M'gladbach,11,FC Augsburg,14,68.9,22.7,8.3,2,0,27.05.2023,2022,34
5,E. Frankfurt,8,SC Freiburg,5,31.4,37.1,31.5,2,1,27.05.2023,2022,34
6,VfL Wolfsburg,7,Hertha BSC,18,93.6,3.6,2.8,1,2,27.05.2023,2022,34
7,VfL Bochum,16,B. Leverkusen,6,13.0,16.5,70.5,3,0,27.05.2023,2022,34
